## Jupyter Notebook: ZoeDepth Testing

This notebook will explore ZoeDepth.

In [3]:
!git clone https://github.com/isl-org/ZoeDepth.git && cd ZoeDepth

Cloning into 'ZoeDepth'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 111 (delta 38), reused 23 (delta 23), pack-reused 42 (from 2)
Receiving objects: 100% (111/111), 4.09 MiB | 3.04 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [17]:
import os
import sys
path = os.path.abspath(os.getcwd())
sys.path.append(os.path.join(path, "ZoeDepth"))
from zoedepth.models.builder import build_model
from zoedepth.utils.config import get_config

In [19]:
# ZoeD_N
conf = get_config("zoedepth", "infer")
model_zoe_n = build_model(conf)

# ZoeD_K
conf = get_config("zoedepth", "infer", config_version="kitti")
model_zoe_k = build_model(conf)

# ZoeD_NK
conf = get_config("zoedepth_nk", "infer")
model_zoe_nk = build_model(conf)

img_size [384, 512]


Using cache found in /Users/egg/.cache/torch/hub/intel-isl_MiDaS_master
/Users/egg/micromamba/envs/stardisty/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Params passed to Resize transform:
	width:  512
	height:  384
	resize_target:  True
	keep_aspect_ratio:  True
	ensure_multiple_of:  32
	resize_method:  minimal
Using pretrained resource url::https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_N.pt


Downloading: "https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_N.pt" to /Users/egg/.cache/torch/hub/checkpoints/ZoeD_M12_N.pt
100%|██████████████████████████████████████| 1.34G/1.34G [01:18<00:00, 18.3MB/s]


RuntimeError: Error(s) in loading state_dict for ZoeDepth:
	Unexpected key(s) in state_dict: "core.core.pretrained.model.blocks.0.attn.relative_position_index", "core.core.pretrained.model.blocks.1.attn.relative_position_index", "core.core.pretrained.model.blocks.2.attn.relative_position_index", "core.core.pretrained.model.blocks.3.attn.relative_position_index", "core.core.pretrained.model.blocks.4.attn.relative_position_index", "core.core.pretrained.model.blocks.5.attn.relative_position_index", "core.core.pretrained.model.blocks.6.attn.relative_position_index", "core.core.pretrained.model.blocks.7.attn.relative_position_index", "core.core.pretrained.model.blocks.8.attn.relative_position_index", "core.core.pretrained.model.blocks.9.attn.relative_position_index", "core.core.pretrained.model.blocks.10.attn.relative_position_index", "core.core.pretrained.model.blocks.11.attn.relative_position_index", "core.core.pretrained.model.blocks.12.attn.relative_position_index", "core.core.pretraine

In [12]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.transforms import ToTensor
from zoedepth.utils.misc import colorize
from zoedepth.models.builder import build_model

# Load your image
image_path = "/Users/egg/Desktop/input/image.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Load ZoeDepth model (CPU-friendly)
device = torch.device("cpu")
model = build_model("zoedepth", device)
model.eval()

# Convert image to tensor
input_tensor = ToTensor()(Image.fromarray(image)).unsqueeze(0).to(device)

# Run depth estimation
with torch.no_grad():
    depth = model(input_tensor).squeeze().cpu().numpy()

# Apply colormap for a pseudo-3D effect
depth_colored = colorize(depth, cmap="inferno")

# Blend depth effect with original image
enhanced_image = cv2.addWeighted(image, 0.6, depth_colored, 0.4, 0)

# Save and display
output_path = "/Users/egg/Desktop/output/cells_3D.jpeg"
cv2.imwrite(output_path, cv2.cvtColor(enhanced_image, cv2.COLOR_RGB2BGR))

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(enhanced_image)
plt.title("Pseudo-3D Enhancement")
plt.axis("off")

plt.show()

/Users/egg/micromamba/envs/stardisty/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/egg/micromamba/envs/stardisty/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <A9A8F989-731E-38CF-AD19-47727B1A070E> /Users/egg/micromamba/envs/stardisty/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/Users/egg/micromamba/envs/stardisty/lib/python3.9/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/egg/micromamba/envs/stardisty/lib/python3.9/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/egg/micromamba/envs/stardisty/lib/python3.9/lib-dynload/../../libjpeg.8.dylib' (no such file), '/Users/egg/micromamba/envs/stardisty/bin/../lib/libjpeg.8.dylib' (no such file), '/usr/local/lib/libjpeg.8.dylib' (no such file), '/usr/lib/libjpeg.8.dylib' (no such file, not in dyld cache)
  warn(f"Fail

ModuleNotFoundError: No module named 'zoedepth'